In [5]:
!pip install catboost
!pip install scikit-learn --upgrade

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.7/dist-packages (0.24.1)


In [6]:
# If you have installation questions, please reach out

import pandas as pd # data storage
import numpy as np  # math and stuff

import catboost as cats # graident boosting 
from catboost import CatBoostRegressor, Pool

import seaborn as sns
import scipy.stats as stats

import datetime
import sklearn  
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import cross_val_score, KFold, train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import accuracy_score, max_error, mean_squared_error, median_absolute_error

import matplotlib.pyplot as plt # plotting utility

# dataframes

In [7]:
# df = pd.read_csv('drive/My Drive/1_lewis_research/core_to_wl_merge/Merged_dataset_inner_imputed_12_21_2020.csv')
df = pd.read_csv('drive/My Drive/1_lewis_research/core_to_wl_merge/OS0_Merged_dataset_imputed_3_1_2021.csv')

# df2 = pd.read_csv('drive/My Drive/1_lewis_research/core_to_wl_merge/offset_test_imputed_2_27_2021.csv')
df2 = pd.read_csv('drive/My Drive/1_lewis_research/core_to_wl_merge/OS1_offset_imputed_test.csv')

df3 = pd.read_csv('drive/My Drive/1_lewis_research/core_to_wl_merge/OS2_offset_imputed_test.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'LiveTime2','ScanTime2', 'LiveTime1','ScanTime1',
              'ref_num', 'API', 'well_name', 'sample_num' ], axis=1)

In [9]:
df = df[df.perm_klink_md >= 0]

df = df[df.USGS_ID != 'E997'] # removing E997

In [10]:
dataset = df[[
       'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
       'RESD', 'PHIN', 'PHID', 
       'GR_smooth', 
       'PE_smooth',
       'perm_klink_md'
]]

dataset.replace('NaN',np.nan, regex=True, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [11]:
X = dataset[[ 'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
        'RESD', 'PHIN', 'PHID', 
        'GR_smooth', 
        'PE_smooth']]

Y = dataset[['perm_klink_md']]

Y_array = np.array(Y.values)

In [12]:
seed = 42 # random seed is only used if you want to compare exact answers with friends 
test_size = 0.25 # how much data you want to withold, .15 - 0.3 is a good starting point

X_train, X_test, y_train, y_test = train_test_split(X.values, Y_array, test_size=test_size)

In [13]:
def catboost_perm(X_train, X_test, y_train, y_test, export_name='TEST.csv', max_iter = 200):
  n_feat = np.shape(X_train)[1]
  
  scaler_X = preprocessing.StandardScaler().fit(X_train, n_feat)
  scaler_y = preprocessing.StandardScaler().fit(y_train)

  X_scaled_train = scaler_X.transform(X_train)
  X_scaled_test  = scaler_X.transform(X_test) # use the same transformer
  Y_scaled_train = scaler_y.transform(y_train)
  
  
  model = CatBoostRegressor(objective='RMSE',
                            task_type='GPU',
                            iterations=max_iter)

  model.fit(X_scaled_train, Y_scaled_train, verbose=max_iter )

  preds = model.predict(X_scaled_test)
  preds_transformed = scaler_y.inverse_transform(preds)


  rmse = mean_squared_error(y_test, preds_transformed, squared=False)
  print("Root Mean Squared Error: %f" % (rmse))
  max = max_error(y_test, preds_transformed)
  print("Max Error: %f" % (max))
  MAE = median_absolute_error(y_test, preds_transformed)
  print("Median Abs Error: %f" % (MAE))

  grid = {'learning_rate': [ 0.05, 0.1, 0.2, 0.3],
        'depth': [ 4, 6, 8, 10],
        'l2_leaf_reg': [ 3, 4, 5, 6, 7, 8]}

  model_grid = CatBoostRegressor(objective='RMSE', 
                                 iterations=max_iter, 
                                 verbose=False)

  # Grid Search
  grid_search_result = model_grid.grid_search(grid, 
                                            X=X_train, 
                                            y=y_train, 
                                            cv=5,
                                            verbose=False)
  
  model2 = CatBoostRegressor(objective='RMSE',
                           depth=grid_search_result['params']['depth'],
                           l2_leaf_reg=grid_search_result['params']['l2_leaf_reg'],
                           learning_rate=grid_search_result['params']['learning_rate'],
                           iterations=max_iter)

  model2.fit(X_scaled_train, Y_scaled_train, verbose=500 )

  preds2 = model2.predict(X_scaled_test)
  preds_transformed2 = scaler_y.inverse_transform(preds2)


  rmse2 = mean_squared_error(y_test, preds_transformed2, squared=False)
  print("Root Mean Squared Error: %f" % (rmse2))
  max2 = max_error(y_test, preds_transformed2)
  print("Max Error: %f" % (max2))
  MAE2= median_absolute_error(y_test, preds_transformed2)
  print("Median Abs Error: %f" % (MAE2))

  x = datetime.datetime.now()

  d = {'target': [Y.columns.values, Y.columns.values],
     'MSE': [rmse, rmse2],
     'MAE': [MAE, MAE2],
     'MaxError': [max, max2], 
     'iter':[max_iter, max_iter],
     'day': [x.day, x.day], 
     'month':[x.month, x.month], 
     'year':[x.year, x.year],
     'model':['catboost', 'catboost'],
     'version':[cats.__version__, cats.__version__ ]}
 
  filepath = 'drive/My Drive/1_lewis_research/analysis/experiments/catboost/catboost_results/'

  results = pd.DataFrame(data=d)

  results.to_csv(filepath+export_name)

  return results

# iterations

In [14]:
iter = 150

# No offset

In [15]:
catboost_perm(X_train, X_test, y_train, y_test,'OS0_perm_cat.csv', iter)

Learning rate set to 0.192011
0:	learn: 0.9367714	total: 5.52ms	remaining: 823ms
149:	learn: 0.1060043	total: 607ms	remaining: 0us
Root Mean Squared Error: 0.040134
Max Error: 0.357909
Median Abs Error: 0.006429

bestTest = 0.01462692179
bestIteration = 34


bestTest = 0.01469638296
bestIteration = 17


bestTest = 0.01523416563
bestIteration = 8


bestTest = 0.01431412143
bestIteration = 5


bestTest = 0.0145611809
bestIteration = 32


bestTest = 0.01486010233
bestIteration = 15


bestTest = 0.01382699793
bestIteration = 8


bestTest = 0.01428213366
bestIteration = 5


bestTest = 0.01432907446
bestIteration = 36


bestTest = 0.01472890233
bestIteration = 15


bestTest = 0.01394694399
bestIteration = 7


bestTest = 0.01457659315
bestIteration = 4


bestTest = 0.01433918057
bestIteration = 46


bestTest = 0.01490825295
bestIteration = 15


bestTest = 0.01352936559
bestIteration = 10


bestTest = 0.01419749648
bestIteration = 5


bestTest = 0.01368539588
bestIteration = 53


bestTest = 0.

,target,MSE,MAE,MaxError,iter,day,month,year,model,version
0,[perm_klink_md],0.040134,0.006429,0.357909,150,17,3,2021,catboost,0.24.4
1,[perm_klink_md],0.039493,0.006558,0.353007,150,17,3,2021,catboost,0.24.4


# offset 1

In [16]:
df2 = df2.drop(['Unnamed: 0', 'Unnamed: 0.1', 'LiveTime2','ScanTime2', 'LiveTime1','ScanTime1',
              'ref_num', 'API', 'well_name', 'sample_num' ], axis=1)

df2 = df2[df2.perm_klink_md >= 0]
df2 = df2[df2.USGS_ID != 'E997'] # removing E997

dataset2 = df2[[
       'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
       'RESD', 'PHIN', 'PHID', 
       'GR_smooth', 
       'PE_smooth',
       'perm_klink_md']]

# Features we will use for prediction
X2 = dataset2[['CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
        'RESD', 'PHIN', 'PHID', 
        'GR_smooth', 
        'PE_smooth']]

# What we are trying to predict
Y = dataset2[['perm_klink_md']]

Y_array2 = np.array(Y.values)

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2.values, Y_array2, test_size=test_size)

In [17]:
catboost_perm(X_train2, X_test2, y_train2, y_test2,'OS1_perm_cat.csv', iter)

Learning rate set to 0.192011
0:	learn: 0.9532782	total: 5.18ms	remaining: 772ms
149:	learn: 0.1011473	total: 606ms	remaining: 0us
Root Mean Squared Error: 0.048914
Max Error: 0.365413
Median Abs Error: 0.005898

bestTest = 0.01241418106
bestIteration = 69


bestTest = 0.01267145327
bestIteration = 40


bestTest = 0.01274347617
bestIteration = 20


bestTest = 0.01340731898
bestIteration = 10


bestTest = 0.01247085411
bestIteration = 69


bestTest = 0.01317503092
bestIteration = 32


bestTest = 0.0134819246
bestIteration = 33


bestTest = 0.01336841457
bestIteration = 11


bestTest = 0.01231787277
bestIteration = 87


bestTest = 0.01214078528
bestIteration = 42


bestTest = 0.01354967265
bestIteration = 11


bestTest = 0.0131785357
bestIteration = 11


bestTest = 0.01257213029
bestIteration = 94


bestTest = 0.01271672273
bestIteration = 32


bestTest = 0.01376442901
bestIteration = 11


bestTest = 0.01318024399
bestIteration = 11


bestTest = 0.01282797121
bestIteration = 85


bestTes

,target,MSE,MAE,MaxError,iter,day,month,year,model,version
0,[perm_klink_md],0.048914,0.005898,0.365413,150,17,3,2021,catboost,0.24.4
1,[perm_klink_md],0.048591,0.005588,0.366956,150,17,3,2021,catboost,0.24.4


# Offset 2

In [18]:
df3 = df3.drop(['Unnamed: 0', 'Unnamed: 0.1', 'LiveTime2','ScanTime2', 'LiveTime1','ScanTime1',
              'ref_num', 'API', 'well_name', 'sample_num' ], axis=1)

df3 = df3[df3.perm_klink_md >= 0]
df3 = df3[df3.USGS_ID != 'E997'] # removing E997

dataset3 = df3[[
       'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
       'RESD', 'PHIN', 'PHID', 
       'GR_smooth', 
       'PE_smooth',
       'perm_klink_md']]

# Features we will use for prediction
X3 = dataset3[['CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
        'RESD', 'PHIN', 'PHID', 
        'GR_smooth', 
        'PE_smooth']]

# What we are trying to predict
Y3 = dataset3[['perm_klink_md']]

Y_array3 = np.array(Y3.values)

X_train3, X_test3, y_train3, y_test3 = train_test_split(X3.values, Y_array3, test_size=test_size)

In [19]:
catboost_perm(X_train3, X_test3, y_train3, y_test3,'OS2_perm_cat.csv', iter)

Learning rate set to 0.192011
0:	learn: 0.9527459	total: 5.26ms	remaining: 784ms
149:	learn: 0.0994988	total: 608ms	remaining: 0us
Root Mean Squared Error: 0.043435
Max Error: 0.365779
Median Abs Error: 0.007953

bestTest = 0.01358391519
bestIteration = 51


bestTest = 0.01346556146
bestIteration = 27


bestTest = 0.01313394944
bestIteration = 17


bestTest = 0.01311138148
bestIteration = 10


bestTest = 0.01339440738
bestIteration = 58


bestTest = 0.01237905755
bestIteration = 52


bestTest = 0.01408573427
bestIteration = 10


bestTest = 0.01371803031
bestIteration = 8


bestTest = 0.01335614783
bestIteration = 78


bestTest = 0.01314903897
bestIteration = 40


bestTest = 0.01396190575
bestIteration = 12


bestTest = 0.01363033856
bestIteration = 8


bestTest = 0.01343631545
bestIteration = 61


bestTest = 0.01325696447
bestIteration = 46


bestTest = 0.01358753472
bestIteration = 26


bestTest = 0.01409242687
bestIteration = 10


bestTest = 0.0133959043
bestIteration = 61


bestTest

,target,MSE,MAE,MaxError,iter,day,month,year,model,version
0,[perm_klink_md],0.043435,0.007953,0.365779,150,17,3,2021,catboost,0.24.4
1,[perm_klink_md],0.041895,0.006195,0.364443,150,17,3,2021,catboost,0.24.4
